In [1]:
import pandas as pd
import os

import scipy.stats
from scipy.stats import hypergeom
import statsmodels.stats.multitest as smm
import matplotlib.pyplot as plt
from itertools import combinations
%pylab inline

import numpy
#import statsmodels.stats.multitest as smm
#import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


In [3]:
#import brain atlas data
ba=pd.read_csv("../data/BrainAtlas/Binary Brain atlas V17_w_120619_HB_data_Jan_10_2020.csv",sep=",")

#import signle cell data (see ipynb for converting data to binary)
sc=pd.read_csv("../data/scRNAseq_Cengen2020/071020_sc_neuron_class_vs_all_DE_WX_binary.csv",sep=',')


#imort curated list of motifs to use (excludes extra unc-3, che-1 motifs, removes repressors mab-9 and cog-1)
ctab=pd.read_csv("../data/motif_information/curated_motifs_071320b_with_neuron_class_expression_per_dataset.csv",sep=',')
ctab=ctab.loc[ctab['exclusion_rule']=='None']
#targetortho data
tfiles=os.listdir("../data/TargetOrtho_data/TargetOrtho_data_genes_corrected/")

#read in single factor analysis output
ba_df=pd.read_csv("../output/072020_single_factor_analysis_data_DE_NB_ba.csv",sep=',')

sc_df=pd.read_csv("../output/072020_single_factor_analysis_data_DE_NB_sc.csv",sep=',')

#current gene ids (make sure using gene public name where available and transcript name if no gene name)
gids=pd.read_csv("../data/c_elegans.canonical_bioproject.current.geneIDs.txt",sep=',',header=None)
gids.columns=['index','WB_GID','gene_name','transcript_name']+list(gids.columns.values)[4:]

In [5]:
len(sc_df.drop_duplicates())

1862

In [6]:
def get_sc_genes(neuron_class):
    return sorted(list(sc.loc[sc[neuron_class]==1]["gene_name"].values))

In [7]:
def get_ba_genes(neuron_class):
    df=ba.loc[ba["class"]==neuron_class]
    
    df=df.transpose()[4:]
    df.columns=list(range(len(df.columns)))
    genes=[]
    for i in range(len(df.columns)):
        g=list(df.index[df[i]==1])
        genes=genes+g
    #if gene is in isoform notation, reduce to main gene name (targetortho doesn't distinguish isoforms)
    correct_these={'inx-1a':'inx-1','inx-1b':'inx-1','inx-18a':'inx-18','inx-18b':'inx-18','inx-10a':'inx-10','inx-10b':'inx-10'}
    genes_corrected=[]
    for gene in genes:
        if gene not in correct_these.keys():
            genes_corrected.append(gene)
        else:
            genes_corrected.append(correct_these[gene])
    genes=list(set(genes_corrected))
    return sorted(list(set(genes)))

In [8]:
def import_TO_data(filename,motif_ID):
    tdata=pd.read_csv("../data/TargetOrtho_data/TargetOrtho_data_genes_corrected/%s" %(filename),sep=',')
    tdata["motif_name"]=motif_ID
    return tdata

def get_TO_df(TF,motif_ID): 
    tdata_df=pd.DataFrame()
    for tfile in tfiles:
        if tfile.startswith("%s_%s" %(TF,motif_ID)):
            tdata_df=tdata_df.append(import_TO_data(tfile,motif_ID))
    tdata_df=tdata_df[["ref_gene_name","Rank","motif_name"]]
    #ref_gene_names_corrected=check_gene_names(list(tdata_df['ref_gene_name'].values))
    #tdata_df['ref_gene_name']=ref_gene_names_corrected
    return tdata_df


def ranksums_test(gene_list, tdata):
    if len(gene_list)==0:
        return 'NaN', 1
    results=tdata.loc[tdata["ref_gene_name"].isin(gene_list)][["ref_gene_name","Rank"]]
    ranks_gene_list=results["Rank"]
    ranks_all_genes=tdata["Rank"]
    rs=scipy.stats.ranksums(ranks_gene_list,ranks_all_genes)
    return [rs[0],rs[1]]

def getGenes_w_motifs(gene_list,tdata):
    r=tdata.loc[tdata["ref_gene_name"].isin(gene_list)][["ref_gene_name","Rank"]]
    r=r.sort_values(ascending=True,by="Rank")
    return(r)



In [9]:
def countMarkers_w_motifs(gene_list, tdataA, tdataB):

    #get list of genes with motif matches for TFA and TFB
    resultsA=tdataA.loc[tdataA["ref_gene_name"].isin(gene_list)][["ref_gene_name","Rank"]]
    resultsB=tdataB.loc[tdataB["ref_gene_name"].isin(gene_list)][["ref_gene_name","Rank"]]

    A=set(list(resultsA["ref_gene_name"]))
    B=set(list(resultsB["ref_gene_name"]))

    A_and_B=A.intersection(B)
    A_not_B=A.difference(B)
    B_not_A=B.difference(A)

    neitherA_norB=set(gene_list)-A-B
   

    return len(A), len(B), len(A_and_B), A, B, A_and_B

In [10]:
def hypergeom_test(k,N,n):
    #total coding genes in genome
    M=20094.0
    #k=number of sucesses
    #N=len(ASE_genes_BA)
    #n=taragetortho data length (whole genome motif matchs per gene)

    pval = hypergeom.sf(k-1, M, n, N)
    expected=N/M*n
    observed=k
    fold_enrichment=observed/expected
    prop_found=k/N
    #print([observed,expected,fold_enrichment, pval])
    return [observed,expected,fold_enrichment, pval]

In [11]:
def multiple_test_correction(pval_col,df):
    FDR=.05
    pvals=list(df[pval_col].values)
    print(pval_col,'pvals',pvals[:10])
    corrected_ps=smm.multipletests(pvals, alpha=FDR, method='fdr_bh', is_sorted=False, returnsorted=False)
    return corrected_ps[1]

def correct(df):
    #add multiple testing correctings for all pvals.
    pval_cols=['hg_pvalAB','rs_pvalAB_A', 'rs_pvalAB_B', 'best_subset_rs_pvalAB_A','best_subset_rs_pvalAB_B']
    for pval_col in pval_cols:
        corrected_pvals=multiple_test_correction(pval_col,df)
        #print('corrected',corrected_pvals[:10])
        df["%s*" %pval_col]=corrected_pvals
    return df

In [12]:
def assign_modelCF(p1,p2,p3,p4,p5):
    c=.05
    #enriched and rankedA and B
    if p1<c and (p2 < c and p3 < c): mod='A-I'
    
    #enriched and not ranked A or B,  but has some subset.
    elif p1<c and (p2 >= c or p3 >= c) and (p4 < c and p5 < c): mod='A-II'
            
    #not enriched, yes ranked sig. 
    elif p1 >= c and (p2 < c and p3 < c): mod='B-I'
        
    #not enriched, not ranked sig, yes subset ranked sig
    elif p1 >= c and (p2 >= c or p3 >= c) and (p4 < c and p5 < c): mod='B-II'
        
    #enriched but not rank sig. overall or in subset
    elif p1 < c and (p2 >= c or p3 >= c)  and (p4 >= c or p5 >= c): mod='C-I'

    #nothing sig.
    elif p1 >= c and (p2 >= c or p3 >= c)  and (p4 >= c or p5 >= c): mod='C-II'
    else:
        mod='error'
    return mod

In [14]:
def label_models(df):
    pval_cols=['hg_pvalAB*','rs_pvalAB_A*', 'rs_pvalAB_B*', 'best_subset_rs_pvalAB_A*','best_subset_rs_pvalAB_B*']
    mods_list=[]
    for p1,p2,p3,p4,p5 in df[pval_cols].values:
        mod=assign_modelCF(p1,p2,p3,p4,p5)
        mods_list.append(mod)
    df["model"]=mods_list
    return df

In [15]:
def add_comb_model(d,df):
    models=[]
    print(d.columns.values)
    for motifA_ID, motifB_ID, neuron_class, model in d[['motifA_ID','motifB_ID','neuron_class','model']].values:
        motifA_mod=df.loc[(df["motif_ID"]==motifA_ID) & (df["neuron_class"]==neuron_class)]["model"].values[0]
        motifB_mod=df.loc[(df["motif_ID"]==motifB_ID) & (df["neuron_class"]==neuron_class)]["model"].values[0]
        comb_model="%s,%s,%s" %(model,motifA_mod,motifB_mod)
        models.append(comb_model)
    d["comb_model"]=models
    return d

In [17]:
def subset_rs(genes_w_motifsA_and_B,tdataA,tdataB):
    pvalA=1
    pvalB=1
    sig_subset_len=0
    if len(genes_w_motifsA_and_B)>0:
       
        #get all combinations of genes with both motifs starting with largest set, 
        #do rank sums test for each motif, if both sig. stop and return that p value and len of gene list.
       
        #sort genes by rankA+B, then test largest subset, second largest and so on until 2 sig. p values found
        j=len(genes_w_motifsA_and_B)
        plist=[]
        jlist=[]
        for l in range(j):
            rsA=ranksums_test(genes_w_motifsA_and_B[:j],tdataA)
            rsB=ranksums_test(genes_w_motifsA_and_B[:j],tdataB)

            if rsA[1] <= .05 and rsB[1] <= .05:
                #print(k, rsA[1], rsB[1])
                pvalA=rsA[1]
                pvalB=rsB[1]
                sig_subset_len=j
                plist.append([pvalA,pvalB])
                jlist.append(j)
                
            j-=1
        if len(plist)>0:
            sum_plist=[sum(i) for i in plist]
            min_p_index=sum_plist.index(min(sum_plist))
            ps=plist[min_p_index]
            pvalA=ps[0]
            pvalB=ps[1]
            sig_subset_len=jlist[min_p_index]
            return pvalA,pvalB, sig_subset_len
        else:
            return pvalA,pvalB, sig_subset_len
    return pvalA,pvalB, sig_subset_len
    
    

In [18]:
def run(data_source,df):
    
    headers="TFA,TFB,motifA_ID,motifB_ID,neuron_class,marker_countAB,\
    exp_motifAB,fcAB,hg_pvalAB,rs_pvalAB_A,rs_pvalAB_B,best_subset_rs_pvalAB_A,best_subset_rs_pvalAB_B,\
    num_genes_in_subset".split(',')
    d=pd.DataFrame()
    
    neuron_classes=sorted(list(df["neuron_class"].drop_duplicates().values))
    for neuron_class in neuron_classes[:]:

        motif_list=list(df.loc[df["neuron_class"]==neuron_class]["motif_ID"].drop_duplicates().values)

        combos=[comb for comb in combinations(motif_list, 2)]

        print(len(combos))
        for c in combos:
            motifA_ID=c[0]
            motifB_ID=c[1]

            TFA=ctab.loc[ctab["motif_ID"]==motifA_ID]["TF_Name"].values[0]
            TFB=ctab.loc[ctab["motif_ID"]==motifB_ID]["TF_Name"].values[0]
            print(neuron_class,TFA,TFB,motifA_ID,motifB_ID)
            
            tdataA=get_TO_df(TFA,motifA_ID)
            tdataB=get_TO_df(TFB,motifB_ID)
            
            if data_source=='sc':
                genes=get_sc_genes(neuron_class)
                
            if data_source=='ba':
                genes=get_ba_genes(neuron_class)
            
            lenA, lenB, len_A_and_B, genes_w_motifsA, genes_w_motifsB, genes_w_motifsA_and_B=countMarkers_w_motifs(genes, tdataA, tdataB)

            rsABgenes_motifA=ranksums_test(genes_w_motifsA_and_B, tdataA)
            rsABgenes_motifB=ranksums_test(genes_w_motifsA_and_B, tdataB)
            
            #test subsets for rsAB_A and rsAB_B
            #do rank sums test on subsets of genes to find most sig. subset.
            rAB_A=getGenes_w_motifs(genes_w_motifsA_and_B,tdataA)
            rAB_B=getGenes_w_motifs(genes_w_motifsA_and_B,tdataB)
    
            #merge rAB_A and rAB_B so can sort by sum of ranks for each gene
            rAB_AB=rAB_A.merge(rAB_B,on="ref_gene_name")
         
            rAB_AB["sum_ranks"]=rAB_AB["Rank_x"]+rAB_AB["Rank_y"]
            rAB_AB=rAB_AB.sort_values(ascending=True,by="sum_ranks")
            genes_sorted_by_sum_ranks=rAB_AB["ref_gene_name"].values
            
        
            #genes_sorted_by_rankAB_A=rAB_A["ref_gene_name"].values
            #genes_sorted_by_rankAB_B=rAB_B["ref_gene_name"].values
            
            #get best p value where both motif A and motf B had signficantly ranked set AB genes. (must be same set of genes)
            #subset_results=subset_rs(genes_sorted_by_rankAB_A,genes_sorted_by_rankAB_B,tdataA,tdataB)
            subset_results=subset_rs(genes_sorted_by_sum_ranks,tdataA,tdataB)

            best_pvalAB_A=subset_results[0]
            best_pvalAB_B=subset_results[1]
            num_genes_in_subset=subset_results[2]
            
            #best_pvalAB_A,num_genes_in_subsetAB_A=subset_rs(genes_sorted_by_rankAB_A,tdataA)
            #best_pvalAB_B,num_genes_in_subsetAB_B=subset_rs(genes_sorted_by_rankAB_B,tdataB)

            
            
            
            hA=hypergeom_test(lenA,len(genes),len(tdataA))
            hB=hypergeom_test(lenB,len(genes),len(tdataB))
            expA_prop=hA[1]/len(genes)
            expB_prop=hB[1]/len(genes)
            expAB_prop=expA_prop*expB_prop
            
            k=len(genes_w_motifsA_and_B)
            N=len(genes)
            
            #estimated size of hypothetical motif A & B targetortho dataset from exp proportion (M=number of coding genes in genome)
            M=20094.0
            n=expAB_prop*M
            
            
            hAB=hypergeom_test(k,N,n)
            
            results=[TFA,TFB, motifA_ID,motifB_ID,neuron_class,hAB[0],
            hAB[1],hAB[2],hAB[3],rsABgenes_motifA[1],rsABgenes_motifB[1],best_pvalAB_A,best_pvalAB_B,
            num_genes_in_subset]
            print(results)
            dictionary = dict(zip(headers, results))
            d=d.append(dictionary,ignore_index=True)
        
    #run multiple testing correction on unqiue dataset and categorize into models
    d=d.drop_duplicates()
    
    d=correct(d)
    d=label_models(d)
    d=add_comb_model(d,df)
    cols=['comb_model','model']+headers+['hg_pvalAB*','rs_pvalAB_A*', 'rs_pvalAB_B*', 'best_subset_rs_pvalAB_A*','best_subset_rs_pvalAB_B*']
    return d[cols]


In [19]:
data_source='sc'
df=sc_df
d=run(data_source,df)
d.to_csv("../output/072020_cofactor_analysis_sc.csv",sep=',')

136
ADA jun-1 ceh-38 M00599_2.00 M03716_2.00
['jun-1', 'ceh-38', 'M00599_2.00', 'M03716_2.00', 'ADA', 82, 41.020350998559984, 1.9990077608765124, 1.809362659628924e-10, 5.4513315691642925e-06, 0.003264845419191543, 8.854522388507366e-09, 8.79687518727098e-09, 60]
ADA jun-1 ceh-48 M00599_2.00 M01905_2.00
['jun-1', 'ceh-48', 'M00599_2.00', 'M01905_2.00', 'ADA', 84, 40.73226914421896, 2.0622470037842695, 1.9259311537297816e-11, 0.0002723132301949702, 0.4945140941940842, 7.285334688491256e-07, 1.5502077532689387e-06, 44]
ADA jun-1 ceh-74 M00599_2.00 M01248_2.00
['jun-1', 'ceh-74', 'M00599_2.00', 'M01248_2.00', 'ADA', 63, 19.071853777243973, 3.3032971380668785, 1.5318354061758556e-17, 2.4927541067405676e-05, 3.2190616184208694e-05, 1.6705673869219865e-10, 1.9533812459883365e-10, 38]
ADA jun-1 ceh-58 M00599_2.00 M00676_2.00
['jun-1', 'ceh-58', 'M00599_2.00', 'M00676_2.00', 'ADA', 77, 27.572356029973555, 2.792652173658801, 2.3863752565837287e-17, 6.0437499541862856e-05, 0.0001343500627527229,

['ceh-48', 'ceh-58', 'M01905_2.00', 'M00676_2.00', 'ADA', 76, 43.56208994792259, 1.7446362213304307, 3.96916763398492e-07, 0.029356238588925238, 0.014729388322746433, 2.6405596425726007e-08, 3.1497461530455666e-08, 27]
ADA ceh-48 unc-62 M01905_2.00 M02185_2.00
['ceh-48', 'unc-62', 'M01905_2.00', 'M02185_2.00', 'ADA', 90, 42.414330461105735, 2.121924335986647, 4.828805995261583e-13, 0.26669201958926836, 1.0162207003257438e-05, 1.5428599373332358e-09, 1.0626119250062652e-09, 43]
ADA ceh-48 ceh-18 M01905_2.00 M00681_2.00
['ceh-48', 'ceh-18', 'M01905_2.00', 'M00681_2.00', 'ADA', 113, 61.22703193467859, 1.8455900348159378, 1.4162298628943618e-12, 0.5446474664209854, 1.4550319943231832e-09, 3.3130932002621686e-10, 9.380990921508839e-15, 53]
ADA ceh-48 unc-86 M01905_2.00 M03312_2.00
['ceh-48', 'unc-86', 'M01905_2.00', 'M03312_2.00', 'ADA', 130, 91.37880557973524, 1.4226493679277161, 7.802213060573871e-07, 0.4231757369295661, 3.544142507985981e-10, 1.8601330702488374e-08, 2.1283746388335353e-1

['ceh-58', 'vab-7', 'M00676_2.00', 'M02186_2.00', 'ADA', 94, 42.57082467850491, 2.208085013853701, 9.242808340517754e-15, 0.00010202268160032222, 1.2835261605804547e-10, 1.8468146626290743e-13, 1.803857845247994e-15, 39]
ADA ceh-58 mbr-1 M00676_2.00 M00707_2.00
['ceh-58', 'mbr-1', 'M00676_2.00', 'M00707_2.00', 'ADA', 47, 15.217261433222651, 3.0885977878640234, 4.29366886303528e-12, 0.0008064042477245297, 0.016877293762065804, 5.619998858197916e-06, 0.0002694791330392716, 12]
ADA ceh-58 lin-31 M00676_2.00 M00665_2.00
['ceh-58', 'lin-31', 'M00676_2.00', 'M00665_2.00', 'ADA', 98, 53.24255438079427, 1.8406329512122488, 1.3881429938132092e-10, 5.574114849715932e-05, 2.7429867300992252e-09, 7.005761093395797e-12, 1.793607492997471e-16, 39]
ADA ceh-58 ztf-3 M00676_2.00 M00640_2.00
['ceh-58', 'ztf-3', 'M00676_2.00', 'M00640_2.00', 'ADA', 84, 29.979076661577725, 2.801954207871167, 3.7914510034928423e-19, 0.0001627291273031415, 4.965604248301858e-06, 1.0916739739661308e-11, 2.31642870149839e-12,

['unc-86', 'lin-31', 'M03312_2.00', 'M00665_2.00', 'ADA', 149, 111.6852068196764, 1.3341068548189283, 3.116645289075282e-06, 5.091276726050419e-17, 2.7423445898535945e-08, 7.978050742520592e-31, 1.2039039276650967e-23, 71]
ADA unc-86 ztf-3 M03312_2.00 M00640_2.00
['unc-86', 'ztf-3', 'M03312_2.00', 'M00640_2.00', 'ADA', 123, 62.886152179411894, 1.955915503449559, 5.945029571492468e-16, 1.4410774319511514e-19, 0.0001760653168620218, 1.094048217341975e-26, 1.295510260060316e-16, 50]
ADA unc-86 ztf-9 M03312_2.00 M00636_2.00
['unc-86', 'ztf-9', 'M03312_2.00', 'M00636_2.00', 'ADA', 100, 49.997956558489825, 2.000081741001066, 4.957276926659512e-13, 4.5835334674743236e-17, 4.255069571458685e-06, 1.767359861509883e-27, 9.776575017933014e-13, 58]
ADA unc-86 ztf-11 M03312_2.00 M00650_2.00
['unc-86', 'ztf-11', 'M03312_2.00', 'M00650_2.00', 'ADA', 151, 120.9111317372696, 1.248851101055866, 0.00014987022394296291, 3.172283822594696e-17, 3.2393351044728064e-14, 1.496331254159529e-34, 1.03013823951410

['mbr-1', 'ztf-9', 'M00707_2.00', 'M00636_2.00', 'ADA', 39, 12.300082000384993, 3.170710569147368, 1.7088576640431308e-10, 0.5160451929995662, 0.0017730597705013986, 0.0002769290990283097, 4.605969991838649e-06, 15]
ADA mbr-1 ztf-11 M00707_2.00 M00650_2.00
['mbr-1', 'ztf-11', 'M00707_2.00', 'M00650_2.00', 'ADA', 66, 29.745552368484432, 2.2188191089007088, 2.9562296222239537e-10, 0.0976339590285654, 5.575899297391421e-10, 6.362046942711768e-05, 1.641993461892968e-09, 16]
ADA lin-31 ztf-3 M00665_2.00 M00640_2.00
['lin-31', 'ztf-3', 'M00665_2.00', 'M00640_2.00', 'ADA', 118, 54.12939280930538, 2.179961641463575, 7.325680709259591e-19, 1.5411940980682615e-06, 0.0009912565966983264, 2.3870939069642582e-18, 8.229034121300766e-17, 46]
ADA lin-31 ztf-9 M00665_2.00 M00636_2.00
['lin-31', 'ztf-9', 'M00665_2.00', 'M00636_2.00', 'ADA', 95, 43.035850285384576, 2.2074619037389622, 6.255244295769457e-15, 1.0774549098846284e-09, 1.5443516408335555e-06, 9.313263797425885e-19, 1.6156363911077602e-13, 37]

...

['ceh-38', 'ztf-3', 'M03716_2.00', 'M00640_2.00', 'VD_DD', 77, 27.773462189637637, 2.7724307280901024, 5.866281553371269e-19, 5.7412768829852545e-06, 5.205734037877536e-13, 1.1163080555625739e-13, 3.1975773621965e-13, 58]
VD_DD ceh-38 ztf-9 M03716_2.00 M00636_2.00
['ceh-38', 'ztf-9', 'M03716_2.00', 'M00636_2.00', 'VD_DD', 66, 22.081433000936208, 2.988936451597219, 1.7909845825064618e-17, 0.0014910808649287353, 4.126020172401351e-06, 3.041636304176711e-09, 5.391822532172914e-08, 49]
VD_DD ceh-38 ztf-11 M03716_2.00 M00650_2.00
['ceh-38', 'ztf-11', 'M03716_2.00', 'M00650_2.00', 'VD_DD', 95, 53.400003486153146, 1.7790261010869393, 3.137469522589377e-11, 6.381633861743984e-06, 2.4177871085059415e-14, 5.548463539765838e-16, 1.7912749442415375e-16, 47]
VD_DD ceh-38 elt-1 M03716_2.00 M08206_2.00
['ceh-38', 'elt-1', 'M03716_2.00', 'M08206_2.00', 'VD_DD', 60, 26.970952260416652, 2.2246155575328994, 4.010691716879038e-10, 0.008090108599361484, 4.513138409329277e-06, 1.0037540628629168e-09, 2.2520

['ceh-74', 'ztf-3', 'M01248_2.00', 'M00640_2.00', 'VD_DD', 69, 12.912893158500225, 5.34349654667274, 3.275258871095337e-33, 0.00032654854583348325, 9.966156906101686e-09, 4.5181378147037836e-12, 5.794479575488158e-12, 33]
VD_DD ceh-74 ztf-9 M01248_2.00 M00636_2.00
['ceh-74', 'ztf-9', 'M01248_2.00', 'M00636_2.00', 'VD_DD', 60, 10.266461674124843, 5.844272535611902, 1.5926981903560528e-30, 0.00010049654293590737, 2.3634503625431446e-05, 4.0377867890437754e-09, 4.0627056720604646e-08, 29]
VD_DD ceh-74 ztf-11 M01248_2.00 M00650_2.00
['ceh-74', 'ztf-11', 'M01248_2.00', 'M00650_2.00', 'VD_DD', 84, 24.827604674274568, 3.383330816727465, 4.5092702112130384e-27, 0.001631931493741893, 1.9426618429757345e-13, 2.733036010330357e-10, 2.379024316027419e-12, 27]
VD_DD ceh-74 elt-1 M01248_2.00 M08206_2.00
['ceh-74', 'elt-1', 'M01248_2.00', 'M08206_2.00', 'VD_DD', 56, 12.539777091662417, 4.46578911177248, 1.5310572931743397e-22, 0.002007543311367926, 3.3704225777886653e-07, 3.1137048924814174e-08, 2.29

['ceh-18', 'vab-3', 'M00681_2.00', 'M00342_2.00', 'VD_DD', 78, 29.665842115660457, 2.629286561153245, 7.193194534940839e-18, 4.067108833743963e-10, 0.00826669359329945, 1.0445561293378334e-11, 6.315874353352236e-12, 40]
VD_DD ceh-18 irx-1 M00681_2.00 M02064_2.00
['ceh-18', 'irx-1', 'M00681_2.00', 'M02064_2.00', 'VD_DD', 55, 15.241310005411115, 3.6086136939983096, 7.580637916341835e-18, 4.818324462725736e-08, 0.08978205746742798, 7.939794992847415e-09, 1.5830063303001339e-06, 15]
VD_DD ceh-18 nsy-7 M00681_2.00 M00966_2.00
['ceh-18', 'nsy-7', 'M00681_2.00', 'M00966_2.00', 'VD_DD', 70, 21.07599611773901, 3.3213139539859364, 2.8523888364656518e-21, 3.114008146106071e-08, 0.8961080064966139, 5.964764766451465e-10, 6.901745791801237e-07, 20]
VD_DD ceh-18 lin-39 M00681_2.00 M08211_2.00
['ceh-18', 'lin-39', 'M00681_2.00', 'M08211_2.00', 'VD_DD', 87, 32.68283989109055, 2.661947379417187, 7.003765153928313e-21, 9.975915307264762e-10, 3.2634032641755973e-15, 5.791024866708218e-17, 1.8317349624479

['irx-1', 'elt-1', 'M02064_2.00', 'M08206_2.00', 'VD_DD', 44, 10.706026851463099, 4.109834638980651, 3.5760023958502967e-16, 0.08316374958440645, 2.8989262879598608e-06, 5.085809712049392e-06, 3.913542743167789e-09, 19]
VD_DD nsy-7 lin-39 M00966_2.00 M08211_2.00
['nsy-7', 'lin-39', 'M00966_2.00', 'M08211_2.00', 'VD_DD', 70, 18.989286310348128, 3.6862891451509587, 6.803812705499052e-24, 0.5771216219945712, 6.93873777937403e-12, 2.640896573584203e-06, 3.446937025599373e-12, 21]
VD_DD nsy-7 ceh-13 M00966_2.00 M00450_2.00
['nsy-7', 'ceh-13', 'M00966_2.00', 'M00450_2.00', 'VD_DD', 63, 15.011140953929388, 4.196882848102817, 3.479493514359882e-24, 0.7833629403357012, 0.01600788985254491, 3.6658212490142825e-05, 1.2539695217918932e-07, 17]
VD_DD nsy-7 egl-5 M00966_2.00 M09581_2.00
['nsy-7', 'egl-5', 'M00966_2.00', 'M09581_2.00', 'VD_DD', 80, 22.867523906674162, 3.498411123412056, 1.8500108616465972e-26, 0.7733089762532572, 2.1334445361919125e-12, 7.003246145770803e-06, 5.635929199044584e-07, 1

['mab-5', 'ztf-3', 'M01240_2.00', 'M00640_2.00', 'VD_DD', 87, 27.620814153175505, 3.1497985366227086, 4.9502512973128575e-26, 3.1152327974484783e-13, 6.918238453965998e-11, 2.0577746051272249e-19, 9.539739994354916e-16, 70]
VD_DD mab-5 ztf-9 M01240_2.00 M00636_2.00
['mab-5', 'ztf-9', 'M01240_2.00', 'M00636_2.00', 'VD_DD', 69, 21.96006939970969, 3.1420665729276873, 1.4092065603556796e-19, 5.41185383055506e-14, 3.1844341170945e-06, 1.1833437348769318e-14, 8.424877421686635e-11, 30]
VD_DD mab-5 ztf-11 M01240_2.00 M00650_2.00
['mab-5', 'ztf-11', 'M01240_2.00', 'M00650_2.00', 'VD_DD', 106, 53.10650728378651, 1.995988917771701, 5.2600053437951404e-17, 4.29276351824662e-13, 1.0378275305785892e-15, 3.321640057594263e-22, 9.54746885530583e-22, 64]
VD_DD mab-5 elt-1 M01240_2.00 M08206_2.00
['mab-5', 'elt-1', 'M01240_2.00', 'M08206_2.00', 'VD_DD', 67, 26.822714965550237, 2.4978828610769446, 8.00822775851784e-14, 6.685226669363024e-11, 2.46419063385268e-06, 1.1007634679731721e-15, 5.82222598716788

In [20]:
len(d.drop_duplicates())

14909

In [18]:
data_source='ba'
df=ba_df
d=run(data_source,df)
d.to_csv("../output/072020_cofactor_analysis_ba.csv",sep=',')

136
ADA jun-1 ceh-38 M00599_2.00 M03716_2.00
['jun-1', 'ceh-38', 'M00599_2.00', 'M03716_2.00', 'ADA', 17, 7.062111710374796, 2.4072119922750113, 0.0002889432576320749, 0.5316380890142158, 0.1512720691119455, 1, 1, 0]
ADA jun-1 ceh-48 M00599_2.00 M01905_2.00
['jun-1', 'ceh-48', 'M00599_2.00', 'M01905_2.00', 'ADA', 19, 7.0125152006530795, 2.709441542205929, 1.9388220357253774e-05, 0.6106935849296358, 0.4557559590503585, 0.04107825213201907, 0.008514167256122118, 3]
ADA jun-1 ceh-74 M00599_2.00 M01248_2.00
['jun-1', 'ceh-74', 'M00599_2.00', 'M01248_2.00', 'ADA', 15, 3.283432701576801, 4.568389659028662, 3.8117217866118797e-07, 0.9409582515809458, 0.07262031291149819, 0.02936459337725026, 0.00389944936760694, 4]
ADA jun-1 ceh-58 M00599_2.00 M00676_2.00
['jun-1', 'ceh-58', 'M00599_2.00', 'M00676_2.00', 'ADA', 17, 4.74688913336541, 3.581292826181403, 1.5680178893029918e-06, 0.7561199977247881, 0.01219442073870338, 0.0439496935618859, 0.0115955164571418, 3]
ADA jun-1 unc-62 M00599_2.00 M02185

ADA ceh-48 ceh-18 M01905_2.00 M00681_2.00
['ceh-48', 'ceh-18', 'M01905_2.00', 'M00681_2.00', 'ADA', 25, 10.540917585823788, 2.371710033443565, 4.170812432722677e-06, 0.1276840973551737, 0.0005827729740931825, 3.522150633067233e-05, 1.0262403490367252e-06, 13]
ADA ceh-48 unc-86 M01905_2.00 M03312_2.00
['ceh-48', 'unc-86', 'M01905_2.00', 'M03312_2.00', 'ADA', 29, 15.731882279295077, 1.8433903512084664, 6.778847688640243e-05, 0.0129940148101155, 2.6109314867543615e-05, 1.6071428774614188e-05, 3.7799058298739175e-08, 19]
ADA ceh-48 unc-86 M01905_2.00 M02244_2.00
['ceh-48', 'unc-86', 'M01905_2.00', 'M02244_2.00', 'ADA', 22, 6.57529765372977, 3.3458561358846968, 6.388817080649886e-08, 0.022500577755498533, 0.005355670557093829, 0.0002038073363024792, 0.00014599523403162137, 17]
ADA ceh-48 ceh-14 M01905_2.00 M00435_2.00
['ceh-48', 'ceh-14', 'M01905_2.00', 'M00435_2.00', 'ADA', 29, 13.108576997755222, 2.2122919982059153, 1.3189326008277103e-06, 0.10517830209353193, 0.0001439497927971574, 4.362

['ceh-58', 'lin-31', 'M00676_2.00', 'M00665_2.00', 'ADA', 26, 9.166300571052492, 2.836476918737417, 5.402592382232995e-08, 0.09718369536187657, 0.002624128277834623, 2.3262898371697894e-06, 1.464118651519811e-05, 12]
ADA ceh-58 ztf-3 M00676_2.00 M00640_2.00
['ceh-58', 'ztf-3', 'M00676_2.00', 'M00640_2.00', 'ADA', 21, 5.16123297836686, 4.068795206110791, 4.78497202898001e-09, 0.007898395719139592, 0.06501666225601332, 7.015494852202525e-05, 5.009110570482452e-05, 9]
ADA ceh-58 ztf-9 M00676_2.00 M00636_2.00
['ceh-58', 'ztf-9', 'M00676_2.00', 'M00636_2.00', 'ADA', 18, 4.103464646786163, 4.386537121526712, 3.096084651978726e-08, 0.008452315589859543, 0.0302565271645236, 6.465243569983606e-05, 0.00032714435947616484, 9]
ADA ceh-58 ztf-11 M00676_2.00 M00650_2.00
['ceh-58', 'ztf-11', 'M00676_2.00', 'M00650_2.00', 'ADA', 26, 9.923496651435476, 2.620044215587959, 2.8661283619342396e-07, 0.03158733250967794, 0.00105309899663661, 1.2872945998581917e-06, 5.091660495249347e-06, 13]
ADA unc-62 ceh-1

['unc-86', 'ztf-9', 'M03312_2.00', 'M00636_2.00', 'ADA', 22, 8.607706806772974, 2.555849135415405, 7.2744082000626765e-06, 0.00017918976070156232, 0.0592159593268533, 3.198683775681363e-06, 4.5768180524404134e-05, 9]
ADA unc-86 ztf-11 M03312_2.00 M00650_2.00
['unc-86', 'ztf-11', 'M03312_2.00', 'M00650_2.00', 'ADA', 34, 20.81620216722224, 1.6333430914471674, 9.063532365504566e-05, 4.917993565823616e-05, 1.9562964261269827e-05, 1.5095080771123485e-08, 1.0779118181508784e-08, 22]
ADA unc-86 ceh-14 M02244_2.00 M00435_2.00
['unc-86', 'ceh-14', 'M02244_2.00', 'M00435_2.00', 'ADA', 22, 7.779690530648087, 2.8278759821269253, 1.2865007276122872e-06, 0.008273077276391101, 8.154123004996588e-05, 2.4991015895778514e-05, 2.439385316961197e-05, 10]
ADA unc-86 vab-3 M02244_2.00 M00342_2.00
['unc-86', 'vab-3', 'M02244_2.00', 'M00342_2.00', 'ADA', 20, 5.11614232159314, 3.9091953942696627, 2.6578511708177862e-08, 0.00010998036628974059, 0.4696266474725854, 0.0009743788886732403, 0.0029818795879112815, 9

['lin-31', 'ztf-9', 'M00665_2.00', 'M00636_2.00', 'ADA', 22, 7.409102430084525, 2.969320536138008, 5.462301428917638e-07, 0.00017001286305552354, 0.13905741172285674, 1.733533296849524e-05, 0.0001869086037186869, 9]
ADA lin-31 ztf-11 M00665_2.00 M00650_2.00
['lin-31', 'ztf-11', 'M00665_2.00', 'M00650_2.00', 'ADA', 34, 17.917591470580955, 1.8975764714707826, 1.9768267460515916e-06, 0.0007201818954748001, 7.23849687990694e-05, 5.177644149587292e-07, 2.7363109231681334e-07, 15]
ADA ztf-3 ztf-9 M00640_2.00 M00636_2.00
['ztf-3', 'ztf-9', 'M00640_2.00', 'M00636_2.00', 'ADA', 19, 4.171814300200227, 4.554373381166101, 5.8113193782901625e-09, 0.04183073381114351, 0.04520126510065461, 0.0001662311795025859, 0.0002268105097373014, 7]
ADA ztf-3 ztf-11 M00640_2.00 M00650_2.00
['ztf-3', 'ztf-11', 'M00640_2.00', 'M00650_2.00', 'ADA', 26, 10.088788085665929, 2.5771182603132083, 4.0355684239310807e-07, 0.12193167949560085, 7.28130462021833e-05, 2.6181541493489176e-05, 4.25922795035639e-05, 11]
ADA ztf-

...

['cebp-1', 'mab-5', 'M00604_2.00', 'M01240_2.00', 'VD', 37, 15.660530702770735, 2.362627467883562, 4.7316622293608e-10, 8.37455955264447e-06, 3.4417584837117466e-06, 4.777412505083473e-10, 6.552726217477548e-10, 17]
VD cebp-1 unc-30 M00604_2.00 M01241_2.00
['cebp-1', 'unc-30', 'M00604_2.00', 'M01241_2.00', 'VD', 27, 8.193289627731449, 3.2953796614993744, 1.054509623984263e-09, 2.6315109786910487e-05, 0.003920904595390969, 2.6799116777784627e-08, 7.604374823998978e-07, 20]
VD cebp-1 ztf-3 M00604_2.00 M00640_2.00
['cebp-1', 'ztf-3', 'M00604_2.00', 'M00640_2.00', 'VD', 30, 10.760905039238843, 2.7878695974555323, 3.44265634306595e-09, 2.5737233229728988e-06, 1.634946502425989e-05, 1.4785166512885805e-09, 5.448765045869538e-08, 21]
VD cebp-1 ztf-9 M00604_2.00 M00636_2.00
['cebp-1', 'ztf-9', 'M00604_2.00', 'M00636_2.00', 'VD', 25, 8.555512525984055, 2.922092618539472, 8.41022598961534e-08, 2.629858320289017e-05, 6.325319234738494e-05, 1.0571867589927749e-08, 5.007797014290799e-07, 15]
VD ceb

['ceh-48', 'ztf-3', 'M01905_2.00', 'M00640_2.00', 'VD', 27, 7.949071671296596, 3.396623041844577, 5.261071025587049e-10, 0.0004780981295520001, 5.7647336786991135e-06, 1.8246674850428752e-07, 3.569686181078689e-08, 18]
VD ceh-48 ztf-9 M01905_2.00 M00636_2.00
['ceh-48', 'ztf-9', 'M01905_2.00', 'M00636_2.00', 'VD', 24, 6.319950041909626, 3.7974983727479272, 9.852460074172556e-10, 0.0018602744504796539, 0.00015023927544348715, 2.6002875319120856e-06, 3.956039731983347e-06, 16]
VD ceh-48 ztf-11 M01905_2.00 M00650_2.00
['ceh-48', 'ztf-11', 'M01905_2.00', 'M00650_2.00', 'VD', 34, 15.283670867555516, 2.2245964529487403, 4.3008090052474e-08, 0.0010764013721879253, 3.847593014511172e-08, 4.6818758044956717e-08, 1.3800085071147836e-08, 17]
VD ceh-48 elt-1 M01905_2.00 M08206_2.00
['ceh-48', 'elt-1', 'M01905_2.00', 'M08206_2.00', 'VD', 22, 7.719384464827793, 2.8499681678299185, 1.3119285103396464e-06, 0.0001258476399101284, 5.760552303266904e-08, 4.7197349598756485e-07, 7.974549415472442e-08, 14]


['unc-62', 'nsy-7', 'M02185_2.00', 'M00966_2.00', 'VD', 21, 4.20828047759486, 4.990161685231123, 1.3466946679505672e-10, 0.00026595496028803284, 0.43985578484492005, 9.355115839438477e-05, 0.0016642683769235915, 8]
VD unc-62 lin-39 M02185_2.00 M08211_2.00
['unc-62', 'lin-39', 'M02185_2.00', 'M08211_2.00', 'VD', 29, 6.525838982778753, 4.443873052419625, 6.59736377955225e-14, 1.043457060424143e-05, 1.5006856285266519e-05, 7.804676688303082e-07, 6.154445672010126e-09, 16]
VD unc-62 ceh-13 M02185_2.00 M00450_2.00
['unc-62', 'ceh-13', 'M02185_2.00', 'M00450_2.00', 'VD', 23, 5.158713561538961, 4.458475882723479, 1.1148129852211661e-10, 1.941136660963608e-05, 0.0034507920552657137, 1.477427475553854e-06, 4.583480206776831e-06, 12]
VD unc-62 egl-5 M02185_2.00 M09581_2.00
['unc-62', 'egl-5', 'M02185_2.00', 'M09581_2.00', 'VD', 26, 7.858630203941746, 3.3084646210937465, 2.5361599887448583e-09, 3.9805789217987476e-06, 1.6359741820156917e-06, 6.064477611815769e-07, 1.8818848256276905e-08, 20]
VD u

['nsy-7', 'egl-5', 'M00966_2.00', 'M09581_2.00', 'VD', 19, 6.591227478982555, 2.8826193695461573, 8.678960577928245e-06, 0.2714587586860203, 3.761693845889284e-06, 0.0003175088024659961, 1.191915637347825e-05, 9]
VD nsy-7 mab-5 M00966_2.00 M01240_2.00
['nsy-7', 'mab-5', 'M00966_2.00', 'M01240_2.00', 'VD', 20, 6.39488468842603, 3.1274997086652068, 1.215395915572342e-06, 0.2597007541262638, 1.9699574321703898e-07, 0.0016520922550824037, 2.680130520555879e-06, 9]
VD nsy-7 unc-30 M00966_2.00 M01241_2.00
['nsy-7', 'unc-30', 'M00966_2.00', 'M01241_2.00', 'VD', 19, 3.345681151083191, 5.6789631593699825, 1.6435606363463731e-10, 0.21002716832481472, 0.0003596879723258851, 0.00032228215203337164, 1.1222755594544324e-05, 10]
VD nsy-7 ztf-3 M00966_2.00 M00640_2.00
['nsy-7', 'ztf-3', 'M00966_2.00', 'M00640_2.00', 'VD', 20, 4.394151652655037, 4.551504267704459, 2.257269624718787e-09, 0.3855952469183104, 9.866221884161023e-06, 0.0004040729118146962, 3.231667321238926e-05, 8]
VD nsy-7 ztf-9 M00966_2.0

['unc-30', 'ztf-3', 'M01241_2.00', 'M00640_2.00', 'VD', 25, 4.165196226288252, 6.00211818166328, 1.2336105729140877e-14, 0.00899319343998961, 3.9205678302896715e-06, 1.8583669720164186e-06, 8.477724758706861e-08, 15]
VD unc-30 ztf-9 M01241_2.00 M00636_2.00
['unc-30', 'ztf-9', 'M01241_2.00', 'M00636_2.00', 'VD', 21, 3.3115605385651907, 6.341421138294736, 1.4372416648900715e-12, 0.005037878889596443, 0.0009820286254579984, 7.915159404604955e-06, 3.716510105116975e-06, 16]
VD unc-30 ztf-11 M01241_2.00 M00650_2.00
['unc-30', 'ztf-11', 'M01241_2.00', 'M00650_2.00', 'VD', 27, 8.008417945361193, 3.3714524122257523, 6.244296980644365e-10, 0.004403432065262108, 9.208787915316366e-06, 7.604374823998978e-07, 7.443637277442492e-08, 20]
VD unc-30 elt-1 M01241_2.00 M08206_2.00
['unc-30', 'elt-1', 'M01241_2.00', 'M08206_2.00', 'VD', 19, 4.04484352031567, 4.69733869915373, 3.9995088143534274e-09, 3.711717710875105e-05, 4.949813297457475e-06, 4.717878727635914e-06, 2.3630961262269172e-07, 15]
VD ztf-3 

In [20]:
len(d.drop_duplicates())

8436